In [1]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import classification_report
import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np

In [2]:
# 기본 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.image_paths = []
        self.labels = []

        normal_dir = os.path.join(data_dir, 'NORMAL')
        for img_name in os.listdir(normal_dir):
            self.image_paths.append(os.path.join(normal_dir, img_name))
            self.labels.append(0)

        pneumonia_dir = os.path.join(data_dir, 'PNEUMONIA')
        for img_name in os.listdir(pneumonia_dir):
            self.image_paths.append(os.path.join(pneumonia_dir, img_name))
            self.labels.append(1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('L')
        label = self.labels[idx]
        return np.array(image), label  # PIL 이미지를 numpy 배열로 변환하여 반환
    

# transform을 적용한 새로운 데이터셋 클래스 정의
class TransformedDataset(Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = Subset(dataset, indices)
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        if self.transform:
            augmented = self.transform(image=image)  # Albumentations transform 적용
            image = augmented['image']
        return image, label



In [3]:

# Albumentations transform 정의
train_transform = A.Compose([
    A.Resize(224, 224),  # 이미지 크기 조정
    A.Rotate(limit=5),  # 최대 5도 회전
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # 밝기 및 대비 조절
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=0, p=0.5),  # 약간의 이동 및 확대/축소
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.2),  # 약간의 가우시안 노이즈 추가
    A.Normalize(mean=(0.5,), std=(0.5,)),  # 정규화
    ToTensorV2()  # 텐서로 변환
])

val_transform = A.Compose([
    A.Resize(224, 224),  # 검증 데이터에는 기본 전처리만 적용
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])

In [4]:

# 원본 데이터셋 로드 (transform 없음)
full_dataset = CustomDataset(data_dir='chest_xray/train')

# 데이터셋 분할 (80% 훈련, 20% 검증)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_indices, val_indices = random_split(full_dataset, [train_size, val_size])


# 각 데이터셋에 transform 적용
train_dataset = TransformedDataset(full_dataset, train_indices.indices, transform=train_transform)
val_dataset = TransformedDataset(full_dataset, val_indices.indices, transform=val_transform)

# DataLoader 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [5]:

class EnhancedCNN(nn.Module): 
    def __init__(self): 
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128*28*28, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x): 
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, 128*28*28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [6]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
criterion = nn.CrossEntropyLoss()
num_epochs = 50
patience = 7
best_val_loss = float('inf')
early_stop_counter = 0


In [7]:

# 모델 학습 루프
for epoch in range(num_epochs): 
    model.train()
    running_loss = 0.0
    all_train_labels = []
    all_train_predictions = [] 
    correct = 0 
    total = 0 

    for images, labels in tqdm.tqdm(train_loader): 
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        _, predicted = torch.max(outputs.data, 1)
        all_train_labels.extend(labels.cpu().numpy())
        all_train_predictions.extend(predicted.cpu().numpy()) 
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    accuracy = 100 * correct / total
    train_report = classification_report(all_train_labels, all_train_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
    train_recall = train_report['weighted avg']['recall']
    print(f'Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss / len(train_loader):.4f}, Train Accuracy: {accuracy:.2f}%, recall: {train_recall:.4f}')

    model.eval()
    val_loss = 0.0
    all_val_labels = []
    all_val_predictions = []
    correct = 0
    total = 0 

    with torch.no_grad(): 
        for images, labels in val_loader: 
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            all_val_labels.extend(labels.cpu().numpy())
            all_val_predictions.extend(predicted.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_report = classification_report(all_val_labels, all_val_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
    val_weighted_avg_recall = val_report['weighted avg']['recall']
    accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss / len(val_loader):.4f}, Valid Accuracy: {accuracy:.2f}%, recall: {val_weighted_avg_recall:.4f}')

    scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break


100%|██████████| 131/131 [00:22<00:00,  5.86it/s]

Epoch [1/50] - Loss: 1.8472, Train Accuracy: 84.49%, recall: 0.8449


Validation Loss: 0.1058, Valid Accuracy: 96.17%, recall: 0.9617


100%|██████████| 131/131 [00:19<00:00,  6.79it/s]

Epoch [2/50] - Loss: 0.1953, Train Accuracy: 92.55%, recall: 0.9255


...

Epoch [20/50] - Loss: 0.1163, Train Accuracy: 95.90%, recall: 0.9590


Validation Loss: 0.0440, Valid Accuracy: 98.37%, recall: 0.9837


100%|██████████| 131/131 [00:22<00:00,  5.91it/s]

Epoch [21/50] - Loss: 0.1079, Train Accuracy: 96.09%, recall: 0.9609


Validation Loss: 0.0439, Valid Accuracy: 98.47%, recall: 0.9847
Early stopping at epoch 21


In [8]:

# 테스트 데이터셋 준비
test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5,), std=(0.5,)),
    ToTensorV2()
])
test_dataset = CustomDataset(data_dir='chest_xray/test')
test_dataset = TransformedDataset(test_dataset, range(len(test_dataset)), transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# 모델 평가 모드로 설정
model.eval()
all_labels = []
all_predictions = []
total = 0
correct = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
        total += labels.size(0) 
        correct += (predicted == labels).sum().item()

# Classification Report 출력 및 Weighted Average Recall 추출
report = classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA'], output_dict=True)
weighted_avg_recall = report['weighted avg']['recall']
print("Classification Report:")
print(classification_report(all_labels, all_predictions, target_names=['NORMAL', 'PNEUMONIA']))
print(f"Weighted Average Recall: {weighted_avg_recall:.4f}")

# 정확도 출력
accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.4f}%')


Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.96      0.49      0.65       242
   PNEUMONIA       0.76      0.99      0.86       398

    accuracy                           0.80       640
   macro avg       0.86      0.74      0.76       640
weighted avg       0.84      0.80      0.78       640

Weighted Average Recall: 0.8000
Accuracy: 80.0000%
